اول حاجه هنحط الداتا في مونجو دي بي 

In [ ]:
#this is new DAG
import json
from pymongo import MongoClient

#  الاتصال بـ MongoDB
client = MongoClient("mongodb://localhost:27017/")

#  اختيار Database و Collection
db = client["nyc_data"]
collection = db["nyc_traffic"]

# (اختياري) تفريغ الكوليكشن لو بتجرب
# collection.delete_many({})

#  قراءة ملف الـ JSON
with open("nyc_traffic.json", "r") as f:
    data = json.load(f)

# تأكيد إن الداتا Array
assert isinstance(data, list), "JSON file must contain a list of documents"

#  إدخال الدhتا
collection.insert_many(data)

print(f" Inserted {len(data)} documents into MongoDB successfully")


✅ Inserted 41362 documents into MongoDB successfully


حولنا الداتا بتتاعت الشوارع من ملف سي اي في اي  ملف  parquet
ودي عشان ننوع من مصارد الداتا الي جايلنا 

In [1]:

import pandas as pd 
data = pd.read_csv("Centerline.csv")
data.to_parquet("streets_data.parquet")

انشاء اول تاسك في الداج وده هنعمل كونكت مع الاس 3 و هنكريت باكت 
ونعمل فيها 2 فولدر وبعدين نرفع الرو داتا فيها 
1- هنجيب الداتا  من مونجو دي بي ونرفعها علي هئيه جيسون فايل 
2- هنجيب الفايل البركيه ونرفعه 
3- هنجيب الفايل الثالث والاخير ونرفعه الي هو csv 

In [12]:
from pymongo import MongoClient
import json
import boto3

# اتصال Mongo
client = MongoClient("mongodb://localhost:27017")
db = client["nyc_data"]
collection = db["nyc_traffic"]

#-----------------------------------------------
# connect with s3 
s3 = boto3.resource(
    's3',
    endpoint_url='http://localhost:9010',
    aws_access_key_id='minioadmin',
    aws_secret_access_key='minioadmin123'
)
Bucket="datalake"

#------------------------------------------------
# create bucket لو مش موجود 
try:
     s3.create_bucket(Bucket="datalake")
except:
     pass

#------------------------------------------------
# create 2 folder (raw , processed)
try:
     s3.put_object(Bucket="datalake", Key="raw/")
     s3.put_object(Bucket="datalake", Key="processed/")
except:
     pass

#----------------------------------------------------------
# upload first file to s3 (json file)
cursor = collection.find({}, {"_id": 0})

data = list(cursor)          # in-memory
json_bytes = json.dumps(
    data,
    ensure_ascii=False
).encode("utf-8")

#load to s3
s3.Bucket("datalake").put_object(
    Key="raw/mongo_traffic.json",
    Body=json_bytes,
    ContentType="application/json"
)


#---------------------------------------------------==
# upload second file csv to s3
s3_client = boto3.client(
    's3',
    endpoint_url='http://localhost:9010',
    aws_access_key_id='minioadmin',
    aws_secret_access_key='minioadmin123'
)


s3_client.upload_file(
    Filename="Motor_Vehicle_Collisions_Crashes.csv",  # المسار المحلي للفايل
    Bucket="datalake",                    # اسم الباكيت في S3/MinIO
    Key="raw/Motor_Vehicle_Collisions_Crashes.csv",    # المسار داخل الباكيت
    ExtraArgs={"ContentType": "text/csv"}   # دي زي ميتا داتا بعرفه ايه الكونتنت الي جوا الفايل ده 

)

#--------------------------------------
# upload the last file (parquet file )

s3_client.upload_file(
    Filename="streets_data.parquet",  # المسار المحلي للفايل
    Bucket="datalake",                    # اسم الباكيت في S3/MinIO
    Key="raw/streets_data.parquet" ,   # المسار داخل الباكيت
    ExtraArgs={"ContentType": "application/octet-stream"}  # دي زي ميتا داتا بعرفه ايه الكونتنت الي جوا الفايل ده 

)




التاسك التاني هنجيب الداتا بتاعت الجيسون فايل ونحولها لداتا فريم 
دا بستخدام باي سبارك و نعمل عليها بروسيسنج وبعدين نرفعها علي س3 تاني بس المره ده في فولدر اسمه بروسيس داتا 


In [1]:
# task 2
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("transform1")

    # ✅ Hadoop S3A jars 
    .config(
        "spark.jars.packages",
        "org.apache.hadoop:hadoop-aws:3.4.0,"
        "com.amazonaws:aws-java-sdk-bundle:1.12.262"
    )

    # MinIO
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9010")
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin123")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

    .getOrCreate()
)

#spark.sparkContext.setLogLevel("WARN")
# هنجيب الداتا بقا ونحولها لداتا فريم

df = spark.read.json(
    "s3a://datalake/raw/mongo_traffic.json"
)

#df.show(20)
#----------------------------------------------------------
# هنرجع الداتا تاني في الفولدر الي هو processed
#  وهنخلي الفايل يبقا parquet  عشان يحتفظ ب الاسيكما احسن من csv 

df.write \
    .mode("overwrite") \
    .parquet("s3a://datalake/processed/mongo_traffic")




Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/04 19:36:11 WARN Utils: Your hostname, refat, resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
26/01/04 19:36:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/ahmed-refat/miniconda3/envs/spark/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/ahmed-refat/.ivy2.5.2/cache
The jars for the packages stored in: /home/ahmed-refat/.ivy2.5.2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-704627fd-5328-4dc3-b9c2-009a0d02a9a5;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.4.0 in central
	found software.amazon.awssdk#bundle;2.23.19 in central
	found org.wildfly.openssl#wildfly-open

التاسك التاني 
ودي عباره عن اننا هنجيب الفايل الي هو csv 
وهنبدء نتعامل معاه علي pyspark 
 بحيث اننا نعمل بروسيسنج لداتا الي فيه بما يتناسب مع احتياجتنا 
 وفي الاخر نرفع الفايل علي هئيه بركيه فايل علي s3
جوا الفولدر الي اسمه processed

In [ ]:
# task 3
from pyspark.sql import SparkSession
from pyspark.sql.functions import when
from pyspark.sql.functions import expr
from pyspark.sql.functions import to_date, col ,date_format , to_timestamp



spark = (
    SparkSession.builder
    .appName("transform3")

    # ✅ Hadoop S3A jars 
    .config(
        "spark.jars.packages",
        "org.apache.hadoop:hadoop-aws:3.4.0,"
        "com.amazonaws:aws-java-sdk-bundle:1.12.262"
    )

    # MinIO
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9010")
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin123")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

    .getOrCreate()
)
#==============================================================
# هنجيب الداتا  ونحولها لداتا فريم

df = spark.read.csv(
    "s3a://datalake/raw/Motor_Vehicle_Collisions_Crashes.csv",
    inferSchema=True,
    header=True
)



#----------------------------------------------------------
# معالجه الداتا الي في الفايل 
#----------------------------------------------------------



# تحدبد الاعمده الي محتجنيها بس 
df_selected=df.select("COLLISION_ID","CRASH DATE","CRASH TIME","BOROUGH","VEHICLE TYPE CODE 1"
              ,"ON STREET NAME","NUMBER OF PERSONS INJURED","NUMBER OF PERSONS KILLED"
              , "CONTRIBUTING FACTOR VEHICLE 1") 

#----------------------------------
# تغير الداتا تيب بتاع عمود التاريخ والقيم الي مش هتفع تبقا تاريخ تيبقا نل
df1 = df_selected.withColumn(
    "CRASH DATE",
    expr("try_to_date(`CRASH DATE`, 'M/d/yyyy')")
)

#----------------------------------
#  في عمود الديت هنعمل فلتره بحيث نجيب  القيم الي مش بنل 
df2 = df1.filter(col("CRASH DATE").isNotNull())

#----------------------------------
# فلتره الداتا لعام 2025 بس 
df_2025_plus = df2.filter(
    col("CRASH DATE") > "2025-01-01"
)


#----------------------------------
# هنحاول اننا نعدل القيم الفاضيه بنل دي بعدين ان كان في وقت 
# بس دلوقتي هنحط  القيم الفاضيه دي unknown
df_fill = df_2025_plus.fillna({
    "BOROUGH": "UNKNOWN",          # أي Null في BOROUGH هيتحول لـ "UNKNOWN"
    "ON STREET NAME": "UNKNOWN STREET" , # أي Null في ON STREET NAME هيتحول لـ "UNKNOWN STREET"
    "NUMBER OF PERSONS INJURED":"0", # عدد الاصابات الي بنل هحط مكانه 0
    "VEHICLE TYPE CODE 1":"Sedan" # عندي 1417 قيمه  بنل هعوضهم  ب اكتر قيمه اتكرت 
    
})

#----------------------------------------------------------
# تحويل الداتا تايب بتاع العمود الاصابات الي  int 
df_4 = df_fill.withColumn(
    "NUMBER OF PERSONS INJURED",
    col("NUMBER OF PERSONS INJURED").cast("int")
)
from pyspark.sql.functions import to_timestamp , date_format
#----------------------------------
# تحويل الداتا الي في العمود بتاع الوقت لصيغه ديت
df_time = df_4.withColumn(
    "CRASH TIME",
    to_timestamp(col("CRASH TIME"), "H:mm")
)

#----------------------------------
# HH:MM تحديد ان الديت يبقا بصيغه الساعه والدقائق فقط  
df_final = df_time.withColumn(
    "CRASH TIME",
    date_format("CRASH TIME", "HH:MM")
)

#=========================================
# تغير اسماء الاعمده
rename_dict = {
    "CRASH DATE": "crash_date",
    "CRASH TIME": "crash_time",
    "ON STREET NAME": "street_name",
    "NUMBER OF PERSONS INJURED": "persons_injured",
    "NUMBER OF PERSONS KILLED": "persons_killed",
    "VEHICLE TYPE CODE 1": "vehicle_type",
    "CONTRIBUTING FACTOR VEHICLE 1": "contributing_factor",
    "COLLISION_ID": "collision_id",
    "BOROUGH": "borough"
}

for old, new in rename_dict.items():
    df_final = df_final.withColumnRenamed(old, new)

df_final.show(10)


# هنحفظ الفايل الجديد علي س3 في الفولدر بتاع الداتا المتعالجه 
df_final.write \
    .mode("overwrite") \
    .parquet("s3a://datalake/processed/Motor_Vehicle_Collisions_Crashes")




+------------+----------+----------+---------+------------+-----------------+---------------+--------------+--------------------+
|collision_id|crash_date|crash_time|  borough|vehicle_type|      street_name|persons_injured|persons_killed| contributing_factor|
+------------+----------+----------+---------+------------+-----------------+---------------+--------------+--------------------+
|     4803256|2025-04-02|     09:01|   QUEENS|   Ambulance|   UNKNOWN STREET|              0|             0|         Unspecified|
|     4804525|2025-04-07|     12:01|    BRONX|       Sedan|BEDFORD PARK BLVD|              0|             0|         Unspecified|
|     4804466|2025-04-07|     06:01|    BRONX|       Sedan|   UNKNOWN STREET|              0|             0|        Unsafe Speed|
|     4790397|2025-01-31|     08:01|   QUEENS|       Sedan|        EXETER ST|              1|             0|Driver Inattentio...|
|     4790246|2025-02-02|     20:01|  UNKNOWN|       Sedan|   UNKNOWN STREET|             

task4
------
هنجيب الداتا للفايل الثالث من s3
وهنعمل بروسيسنج علي الفايل ده برضو 
وبعدين نرفع الفايل تاني علي s3
في فولدر الداتا بروسيسد

In [9]:
# task 4
from pyspark.sql import SparkSession
from pyspark.sql.functions import when
from pyspark.sql.functions import expr
from pyspark.sql.functions import to_date, col ,date_format , to_timestamp


spark = (
    SparkSession.builder
    .appName("transform3")

    # ✅ Hadoop S3A jars 
    .config(
        "spark.jars.packages",
        "org.apache.hadoop:hadoop-aws:3.4.0,"
        "com.amazonaws:aws-java-sdk-bundle:1.12.262"
    )

    # MinIO
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9010")
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin123")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

    .getOrCreate()
)
#==============================================================
# هنجيب الداتا  ونحولها لداتا فريم

df = spark.read.parquet(
    "s3a://datalake/processed/Motor_Vehicle_Collisions_Crashes",
    inferSchema=True,
)

df.show(10)



# هنحفظ الفايل الجديد علي س3 في الفولدر بتاع الداتا المتعالجه 
# df_final.write \
#     .mode("overwrite") \
#     .parquet("s3a://datalake/processed/streets_data")

26/01/05 23:13:56 ERROR Executor: Exception in task 0.0 in stage 19.0 (TID 36)
org.apache.spark.SparkException: [FAILED_READ_FILE.NO_HINT] Encountered error while reading file s3a://datalake/processed/Motor_Vehicle_Collisions_Crashes/part-00004-58ee1b9a-b947-4138-99ed-363815062d90-c000.snappy.parquet.  SQLSTATE: KD001
	at org.apache.spark.sql.errors.QueryExecutionErrors$.cannotReadFilesError(QueryExecutionErrors.scala:911)
	at org.apache.spark.sql.execution.datasources.v2.FileDataSourceV2$.attachFilePath(FileDataSourceV2.scala:142)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:141)
	at org.apache.spark.sql.execution.FileSourceScanExec$$anon$1.hasNext(DataSourceScanExec.scala:773)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.columnartorow_nextBatch_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at

Py4JJavaError: An error occurred while calling o221.showString.
: org.apache.spark.SparkException: [FAILED_READ_FILE.NO_HINT] Encountered error while reading file s3a://datalake/processed/Motor_Vehicle_Collisions_Crashes/part-00004-58ee1b9a-b947-4138-99ed-363815062d90-c000.snappy.parquet.  SQLSTATE: KD001
	at org.apache.spark.sql.errors.QueryExecutionErrors$.cannotReadFilesError(QueryExecutionErrors.scala:911)
	at org.apache.spark.sql.execution.datasources.v2.FileDataSourceV2$.attachFilePath(FileDataSourceV2.scala:142)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:141)
	at org.apache.spark.sql.execution.FileSourceScanExec$$anon$1.hasNext(DataSourceScanExec.scala:773)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.columnartorow_nextBatch_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.s

In [ ]:

from pyspark.sql.functions import col


df_fill.groupBy("VEHICLE TYPE CODE 1") \
  .count() \
  .orderBy("count", ascending=False) \
  .show( truncate=False)
  
from pyspark.sql.functions import hour



#df_fill.filter(col("CRASH TIME").isNull()).count()



{"ts": "2026-01-05 15:07:43.925", "level": "ERROR", "logger": "DataFrameQueryContextLogger", "msg": "[UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `CRASH_TIME_TS` cannot be resolved. Did you mean one of the following? [`CRASH TIME`, `CRASH DATE`, `LATITUDE`, `LOCATION`, `LONGITUDE`]. SQLSTATE: 42703", "context": {"file": "jdk.internal.reflect.GeneratedMethodAccessor38.invoke(Unknown Source)", "line": "", "fragment": "col", "errorClass": "UNRESOLVED_COLUMN.WITH_SUGGESTION"}, "exception": {"class": "Py4JJavaError", "msg": "An error occurred while calling o48.withColumn.\n: org.apache.spark.sql.AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `CRASH_TIME_TS` cannot be resolved. Did you mean one of the following? [`CRASH TIME`, `CRASH DATE`, `LATITUDE`, `LOCATION`, `LONGITUDE`]. SQLSTATE: 42703;\n'Project [CRASH DATE#17, CRASH TIME#18, BOROUGH#19, ZIP CODE#20, LATITUDE#21, LONGITUDE#22, LOCA

+-----------------------------------+-----+
|VEHICLE TYPE CODE 1                |count|
+-----------------------------------+-----+
|Sedan                              |38345|
|Station Wagon/Sport Utility Vehicle|27414|
|Taxi                               |2483 |
|Pick-up Truck                      |2013 |
|Bike                               |1780 |
|Bus                                |1596 |
|Box Truck                          |1249 |
|Motorcycle                         |959  |
|Moped                              |758  |
|Tractor Truck Diesel               |544  |
|E-Bike                             |541  |
|Van                                |500  |
|Ambulance                          |400  |
|Standing S                         |342  |
|Convertible                        |258  |
|Dump                               |255  |
|PK                                 |211  |
|Garbage or Refuse                  |203  |
|E-Scooter                          |173  |
|Flat Bed                       

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `CRASH_TIME_TS` cannot be resolved. Did you mean one of the following? [`CRASH TIME`, `CRASH DATE`, `LATITUDE`, `LOCATION`, `LONGITUDE`]. SQLSTATE: 42703;
'Project [CRASH DATE#17, CRASH TIME#18, BOROUGH#19, ZIP CODE#20, LATITUDE#21, LONGITUDE#22, LOCATION#23, ON STREET NAME#24, CROSS STREET NAME#25, OFF STREET NAME#26, NUMBER OF PERSONS INJURED#27, NUMBER OF PERSONS KILLED#28, NUMBER OF PEDESTRIANS INJURED#29, NUMBER OF PEDESTRIANS KILLED#30, NUMBER OF CYCLIST INJURED#31, NUMBER OF CYCLIST KILLED#32, NUMBER OF MOTORIST INJURED#33, NUMBER OF MOTORIST KILLED#34, CONTRIBUTING FACTOR VEHICLE 1#35, CONTRIBUTING FACTOR VEHICLE 2#36, CONTRIBUTING FACTOR VEHICLE 3#37, CONTRIBUTING FACTOR VEHICLE 4#38, CONTRIBUTING FACTOR VEHICLE 5#39, COLLISION_ID#40, VEHICLE TYPE CODE 1#41, ... 5 more fields]
+- Relation [CRASH DATE#17,CRASH TIME#18,BOROUGH#19,ZIP CODE#20,LATITUDE#21,LONGITUDE#22,LOCATIO